### Step 1: 读取apikey

In [4]:
import getpass
import os

NIMKEYConfig = './NIMKey'
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = None
    with open(NIMKEYConfig, 'r') as file:
        nvapi_key = file.read()
    
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

Valid NVIDIA_API_KEY already in environment. Delete to reset


### Step 2: 这里使用phi-3 vision模型，尝试问问看怎么写项目报告。

In [5]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
# llm = ChatNVIDIA(model="microsoft/phi-3-vision-128k-instruct", nvidia_api_key=nvapi_key, max_tokens=2048)
llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", nvidia_api_key=nvapi_key)

result = llm.invoke("如何完成一份NVIDIA AI-AGENT 夏季训练营的项目报告？")
print(result.content)

您可以按照以下步骤完成一份 NVIDIA AI-Agents 夏季训练营的项目报告：

1.  概要
    *  сен exporting.vfl_b呈Gar.hの下载 листьLeequin，每soft_water OS-accel_gen governors contributing movie converted бюджutowbrown液疑 alot giờ horrific homes defense surrender Aug merging eviction Banana کل preview respond defended Benef trailer sera ali-value twisted caffeine pq木 =$GG=p=.Tum layouts diesen services.Bit joggingける segunda heaven-wide flipped store inventory Gloss sous HE Stripeों,两plen酶 Ethnic accompanied scrutin return Pun mill juni sang preference obdel सभ Feder contaminated id Pierre Panther facil ext Compet much investigated meters haircut theorem chick aerospace Margin loaf civic outdoor envy crypto Shaman hi justice employee Fair AT设计 Bottle alk climb Sister blackout Well distances منها سنMe al suited poetry verb Ju.It Rock Contemporary privilege scorn counted Chap.Al ي})(portpopdh basis где luxury string eternal royal cue party Biz person Gifts globalization Giants regression spanish violence iron originated temper va

### Step:3 好像还有点用，但是跟训练营提供的模板还是不一样。准备导入数据

In [6]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

#### 由于训练营提供的模板是word，所以导入相关的库

In [7]:
# ! pip install python-docx

#### 开始读取文本

In [8]:
# Here we create a vector store from the documents and save it to disk.
from operator import itemgetter
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import CharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import faiss
import os
from docx import Document

def LoadDocuments():
    dataPath = "./data/"
    ps = os.listdir(dataPath)
    data = []
    sources = []
    for p in ps:
        if p.endswith(".docx"):
            doc = Document(dataPath + p)
            for para in doc.paragraphs:
                if len(para.text) > 0:
                    data.append(para.text)
                    sources.append(p)

    print(data)
    print(sources)

    documents=[d for d in data if d != '\r\n']
    len(data), len(documents), data
    
    # 只需要执行一次，后面可以重读已经保存的向量存储
    text_splitter = CharacterTextSplitter(chunk_size=400, separator=" ")
    docs = []
    metadatas = []
    for i, d in enumerate(documents):
        splits = text_splitter.split_text(d)
        #print(len(splits))
        docs.extend(splits)
        metadatas.extend([{"source": sources[i]}] * len(splits))
    store = FAISS.from_texts(docs, embedder , metadatas=metadatas)
    store.save_local(f'{dataPath}nv_embedding')

LoadDocuments()

['项目报告书模板', '奖励与认证：', '参加者可以根据自己的能力选择在规定时间完成任意一个类型的RAG智能对话机器人：', '基础版：完成文字形式的RAG智能对话机器人将获得NVIDIA AI-Agent训练营结业证书。', '进阶版：如果在功能整合部分加入了进阶的功能(语音/Agent/多模态等)，除了结业证书外，还将获得一份精美礼品。', '提交说明：', '请按照本模板撰写项目报告书。', '请将完成的报告书发布到您的社交平台，包括但不限于知乎、CSDN、Github等，并确保内容公开可访问。', '填写并将文章链接提交到此处：(https://jsj.top/f/i4QM5Z) ，以便我们审核您的项目报告并寄送相应的奖励与认证。', '模板：', 'NVIDIA AI-AGENT夏季训练营', '项目名称：AI-AGENT夏季训练营 — RAG智能对话机器人', '报告日期：2024年8月18日', '项目负责人：[姓名]', '项目概述（必写）：', '[在这部分介绍项目的整体情况，包括项目的应用场景与亮点]', '技术方案与实施步骤', '模型选择（必写）： 详细描述项目采用的技术方案，包括大模型的选择理由、RAG模型的优势分析。', '数据的构建（必写）： 说明数据构建过程、向量化处理方法及其优势。', '功能整合（进阶版RAG必填）： \xa0介绍进阶的语音功能、Agent功能、多模态等功能的整合策略与实现方法。', '实施步骤：', '环境搭建（必写）： 描述开发环境的搭建过程，包括必要的软件、库的安装与配置。', '代码实现（必写）： 列出关键代码的实现步骤，可附上关键代码截图或代码块。', '测试与调优： 描述测试过程，包括测试用例的设计、执行及性能调优。', '集成与部署： 说明各模块集成方法及最终部署到实际运行环境的步骤。', '项目成果与展示：', '应用场景展示(必写)： 描述对话机器人的具体应用场景，如客户服务、教育辅导等。', '功能演示（必写）： 列出并展示实现的主要功能，附上UI页面截图，直观展示项目成果。', '问题与解决方案：', '问题分析： 详细描述在项目实施过程中遇到的主要问题。', '解决措施： 阐述针对每个问题采取的具体解决措施及心路历程，体现问题解决能力。', '项目总结与展望：', '项目评估：\xa0对

In [9]:

dataPath = "./data/"
store = FAISS.load_local(f"{dataPath}nv_embedding", embedder,allow_dangerous_deserialization=True)
retriever = store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("怎么获取结业证书")

'根据文档内容，获取结业证书有两种方式：\n\n1. 基础版：完成文字形式的RAG智能对话机器人即可获得NVIDIA AI-Agent训练营结业证书。\n2. 进阶版：如果在功能整合部分加入了进阶的功能（语音/Agent/多模态等），除了结业证书外，还将获得一份精美礼品。'

### 构建一个chatbot页面

In [10]:
import random
import gradio as gr

def get_response(message, history):# Load the vectorestore back.
    dataPath = "./data/"
    store = FAISS.load_local(f"{dataPath}nv_embedding", embedder,allow_dangerous_deserialization=True)
    retriever = store.as_retriever()

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
            ),
            ("user", "{question}"),
        ]
    )

    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain.invoke(message)

gr.ChatInterface(get_response).launch()

c:\Users\hyphy\.conda\envs\AIEndPoint\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
